In [1]:
from appium import webdriver
from appium.webdriver.common.appiumby import AppiumBy
from appium.options.mac import Mac2Options
import time
import xml.etree.ElementTree as ET

In [97]:
from appium.webdriver.common.touch_action import TouchAction
action = TouchAction(driver)

In [139]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [128]:
def save(file, data):
    with open(file+".xml", "w") as f:
        f.write(data)

In [117]:
options = Mac2Options()
options.bundle_id = 'com.tencent.WeWorkMac'  # 指定企业微信
options.platform_name = 'mac'
options.automation_name = 'mac2'
options.set_capability("autoLaunch", False)
options.set_capability("forceAppLaunch", False)  # 不强制启动应用
options.set_capability("useRunningApp", True)  # 连接已经运行的企业微信
options.set_capability("shouldUseSingletonTestManager", True)  # 避免重复创建 WDA 实例
options.set_capability("wdaLocalPort", 8100)  # 绑定已有 WDA，防止重启应用
# 绑定已有进程id
options.set_capability("connectHardwareKeyboard", True)  # 连接但不干扰输入法
options.set_capability("newCommandTimeout", 6000) # 设置等待新命令的时间
# **初始化 driver**
driver = webdriver.Remote('http://localhost:4723', options=options)

In [147]:
def search_and_click_first_result_1(driver, search_text):    
    # 点击 Search 菜单项
    search_menu = driver.find_element(AppiumBy.XPATH, '//XCUIElementTypeMenuItem[@title="Search"]')
    search_menu.click()
    time.sleep(0.5)
    
    search_actions = ActionChains(driver)
    # 确保先点击搜索框，再模拟 Cmd+A 或 Ctrl+A（全选）
    # 执行指针移动后进行组合键操作
    search_actions.move_to_element(search_menu).click().perform()  # 先点击一下（确保指针事件）
    time.sleep(0.05)  # 延迟，模拟人类操作
    
    # 模拟 Cmd + A（全选）或者 Ctrl + A
    search_actions.key_down(Keys.COMMAND).send_keys('a').key_up(Keys.COMMAND).perform()  # macOS 上的 Cmd + A
    # 如果是 Windows，使用下面的代码：
    # actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()  # Windows 上的 Ctrl + A
    time.sleep(0.05)  # 延迟，模拟人类操作

    # 然后模拟按下 Backspace 清空内容
    search_box.send_keys(Keys.BACKSPACE)  # 清空搜索框
    time.sleep(0.05)  # 延迟，模拟人类操作

    # 输入当前用户
    # search_box.send_keys(current_user)
    # time.sleep(0.5)  # 延迟，模拟用户输入后的思考时间

    # # 模拟按下回车键，选中第一个搜索结果
    # search_box.send_keys(Keys.RETURN)
    # time.sleep(0.2)  # 延迟，模拟用户等待

    # # 再按回车确认选择（如果需要的话）
    # search_box.send_keys(Keys.RETURN)
    # time.sleep(1)  # 延迟，模拟动作后的等待
    
    # 输入搜索文本
    driver.execute_script('macos: keys', {'keys': list(search_text)})
    time.sleep(1)
    search_source = driver.page_source
    save("search", search_source)

    search_menu.send_keys(Keys.RETURN)
    time.sleep(0.2)  # 延迟，模拟用户等待
    # 找到并点击第一个搜索结果
    # first_result = driver.find_element(AppiumBy.XPATH, f'//XCUIElementTypeStaticText[@value="{search_text}"]')
    # first_result.click()

In [164]:
def search_and_click_first_result(driver, search_text):
    applescript = f'''
    tell application "System Events"
        tell process "企业微信"
            activate
            set frontmost to true
            keystroke "f" using {{command down}}
            delay 0.05
            keystroke "a" using {{command down}}
            delay 0.05
            key code 51
            delay 0.05
            keystroke "{search_text}"
            delay 0.2
            keystroke return
            delay 0.2
            keystroke return
            delay 0.2
        end tell
    end tell
    '''
    driver.execute_script("macos: appleScript", {"script": applescript})

In [165]:
search_and_click_first_result(driver, "seanzhang")

WebDriverException: Message: An unknown server-side error occurred while processing the command. Original error: Potentially insecure feature 'apple_script' has not been enabled. If you want to enable this feature and accept the security ramifications, please do so by following the documented instructions at http://appium.io/docs/en/2.0/guides/security/
Stacktrace:
UnknownError: An unknown server-side error occurred while processing the command. Original error: Potentially insecure feature 'apple_script' has not been enabled. If you want to enable this feature and accept the security ramifications, please do so by following the documented instructions at http://appium.io/docs/en/2.0/guides/security/
    at getResponseForW3CError (/Users/edenz/.xnpm/lib/node_modules/appium/node_modules/@appium/base-driver/lib/protocol/errors.js:1067:9)
    at asyncHandler (/Users/edenz/.xnpm/lib/node_modules/appium/node_modules/@appium/base-driver/lib/protocol/protocol.js:485:57)

In [124]:
# 模拟 Cmd+F 快捷键

time.sleep(0.2)

# 输入搜索内容
driver.execute_script('macos: keys', {'keys': list('seanzhang')})
time.sleep(0.2)

page_source = driver.page_source
with open("root.xml", "w") as f:
    f.write(page_source)
root = ET.fromstring(page_source)

tables = root.findall(".//XCUIElementTypeTable")

AttributeError: 'int' object has no attribute 'id'

In [91]:
users = []
# 定位到 Table
for row in tables[0].findall("./XCUIElementTypeTableRow"):
    cell = row.find("./XCUIElementTypeCell")
    if cell is not None:
        static_text = cell.find("./XCUIElementTypeStaticText")
        if static_text is not None and "value" in static_text.attrib:
            users.append(static_text)

In [92]:
sean = None          
for u in users:
    print(u.attrib["value"])
    if 'seanzhang' in u.attrib["value"].lower():
        sean = u
        break

seanzhang(张建昌)


In [69]:
def find_labels(root):
    # 获取第一个 Table（标签菜单）
    label_table = root.find(".//XCUIElementTypeTable")
    labels = []
    
    # 遍历所有行
    for row in label_table.findall("./XCUIElementTypeTableRow"):
        cell = row.find("./XCUIElementTypeCell")
        if cell is not None:
            # 查找每个单元格中的文本
            static_text = cell.find("./XCUIElementTypeStaticText[@value]")
            if static_text is not None and static_text.attrib["value"]:
                labels.append(static_text)
    return labels

In [70]:
def create_new_tag(driver, tag):
    # 解析当前页面的 XML
    page_source = driver.page_source
    root = ET.fromstring(page_source)
    
    # 找到输入框并输入标签值
    input_field = root.find(".//XCUIElementTypeTextField[@placeholderValue='Enter a tag']")
    if input_field is not None:
        print("输入新标签：%s" % tag)
        x = int(input_field.attrib["x"]) + int(input_field.attrib["width"]) / 2
        y = int(input_field.attrib["y"]) + int(input_field.attrib["height"]) / 2
        
        # 点击输入框
        driver.execute_script('macos: click', {'x': x, 'y': y, 'duration': 100})
        # 输入标签值
        driver.execute_script('macos: keys', {'keys': list(tag)})
    
    # 找到并点击 OK 按钮
    ok_button = root.find(".//XCUIElementTypeButton[@title='OK']")
    if ok_button is not None:
        print("创建新标签: %s" % tag)
        x = int(ok_button.attrib["x"]) + int(ok_button.attrib["width"]) / 2
        y = int(ok_button.attrib["y"]) + int(ok_button.attrib["height"]) / 2
        driver.execute_script('macos: click', {'x': x, 'y': y, 'duration': 100})

In [98]:
def set_tag_1(driver, expect_tag):
    # 解析标签菜单
    page_source = driver.page_source
    with open("tags.xml", "w") as f:
        f.write(page_source)
    root = ET.fromstring(page_source)
    # 获取第一个 Table（标签菜单）
    tag_table = root.find(".//XCUIElementTypeTable")
    
    # 先找 New Tag 的位置（以备后用）
    new_tag = None
    for row in tag_table.findall("./XCUIElementTypeTableRow"):
        cell = row.find("./XCUIElementTypeCell")
        if cell is not None:
            static_text = cell.find("./XCUIElementTypeStaticText[@value='New Tag']")
            if static_text is not None:
                new_tag = cell
                break
    
    # 查找期望的标签
    target_tag = None
    for row in tag_table.findall("./XCUIElementTypeTableRow"):
        cell = row.find("./XCUIElementTypeCell")
        if cell is not None:
            static_text = cell.find("./XCUIElementTypeStaticText")
            if static_text is not None and "value" in static_text.attrib:
                if static_text.attrib["value"] == expect_tag:
                    target_tag = cell
                    break
    
    # 如果找到目标标签就点击它，否则点击 New Tag
    if target_tag is not None:
        x = int(target_tag.attrib["x"]) + int(target_tag.attrib["width"]) / 2
        y = int(target_tag.attrib["y"]) + int(target_tag.attrib["height"]) / 2
        print("click %s, x: %d, y: %d" % (expect_tag, x, y))
        #driver.execute_script('macos: click', {'x': x, 'y': y})
        #driver.execute_script('macos: click', {'x': x, 'y': y, 'duration': 200})
        action.tap(x=x, y=y).perform()
    elif new_tag is not None:
        print("click new tag")
        x = int(new_tag.attrib["x"]) + int(new_tag.attrib["width"]) / 2
        y = int(new_tag.attrib["y"]) + int(new_tag.attrib["height"]) / 2
        #driver.execute_script('macos: click', {'x': x, 'y': y})
        driver.execute_script('macos: click', {'x': x, 'y': y, 'duration': 200})
        create_new_tag(driver, expect_tag)
    else:
        print("未找到 New Tag 按钮")

In [110]:
def set_tag(driver, expect_tag):
    # 解析标签菜单
    page_source = driver.page_source
    with open("tags.xml", "w") as f:
        f.write(page_source)
    root = ET.fromstring(page_source)
    # 获取第一个 Table（标签菜单）
    tag_table = root.find(".//XCUIElementTypeTable")
    
    # 先找 New Tag 的位置（以备后用）
    new_tag = None
    for row in tag_table.findall("./XCUIElementTypeTableRow"):
        cell = row.find("./XCUIElementTypeCell")
        if cell is not None:
            static_text = cell.find("./XCUIElementTypeStaticText[@value='New Tag']")
            if static_text is not None:
                new_tag = cell
                break
    
    # 查找期望的标签
    target_tag = None
    for row in tag_table.findall("./XCUIElementTypeTableRow"):
        cell = row.find("./XCUIElementTypeCell")
        if cell is not None:
            static_text = cell.find("./XCUIElementTypeStaticText")
            if static_text is not None and "value" in static_text.attrib:
                if static_text.attrib["value"] == expect_tag:
                    target_tag = cell
                    break
    new_x = int(new_tag.attrib["x"]) + int(new_tag.attrib["width"]) / 2
    new_y = int(new_tag.attrib["y"]) + int(new_tag.attrib["height"]) / 2

    # 如果找到目标标签就点击它，否则点击 New Tag
    if target_tag is not None:
        x = int(target_tag.attrib["x"]) + int(target_tag.attrib["width"]) / 2
        y = int(target_tag.attrib["y"]) + int(target_tag.attrib["height"]) / 2
        driver.execute_script('macos: hover', {'x': new_x, 'y': new_y})
        print("click %s, x: %d, y: %d" % (expect_tag, x, y))
        driver.execute_script('macos: click', {'x': x, 'y': y})
    elif new_tag is not None:
        print("click new tag")
        driver.execute_script('macos: click', {'x': new_x, 'y': new_y})
        create_new_tag(driver, expect_tag)
    else:
        print("未找到 New Tag 按钮")

In [116]:
x = int(sean.attrib["x"]) + int(sean.attrib["width"]) / 2
y = int(sean.attrib["y"]) + int(sean.attrib["height"]) / 2
driver.execute_script('macos: rightClick', {'x': x, 'y': y})
menu_source = driver.page_source
menu_root = ET.fromstring(menu_source)
chat_label = menu_root.find(".//XCUIElementTypeStaticText[@value='Chat label']")
if chat_label is not None:
    # 计算元素中心点
    x = int(chat_label.attrib["x"]) + int(chat_label.attrib["width"]) / 2  # 1152 + 67/2
    y = int(chat_label.attrib["y"]) + int(chat_label.attrib["height"]) / 2  # 761 + 22/2
    
    # 点击该位置
    driver.execute_script('macos: click', {'x': x, 'y': y})
    set_tag(driver, "z4")
    time.sleep(0.2)
    # label_source = driver.page_source
    # with open('labels.xml', 'w') as mf:
    #     mf.write(label_source)
    # label_root = ET.fromstring(label_source)
    # labels = find_labels(label_root)
    # for l in labels:
    #     print("label: %s" % l.attrib["value"])
    # expect_label = "xxx"
    

click z4, x: 1355, y: 237


In [115]:
with open('menu.xml', 'w') as mf:
    mf.write(menu_source)